In [2]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import StandardScaler
from pyod.models.knn import KNN

In [3]:
data = pd.read_feather('./base/gs.feather')

In [4]:
def rfm_variables(df):
    f_sales = len(df)
    f_profit = len(df[df['Profit'] > 0])
    m_sales = round(df['Sales'].sum(),2)
    m_profit = round(df['Profit'].sum(),2)
    m_qty = df['Quantity'].sum()
    m_deliver = round(df['Shipping Cost'].sum(),2)
    df_sort = df[['Order Date']].sort_values(by='Order Date').drop_duplicates()
    df_sort['diff'] = df_sort['Order Date'] - df_sort['Order Date'].shift(1)
    df_sort['diff_int'] = df_sort['diff'].dt.days
    r_days = round(df_sort['diff_int'].mean(),2)
    return f_sales, f_profit, m_sales, m_profit, m_qty, m_deliver, r_days

In [5]:
def fit_data(data, variable):
    rfm = pd.DataFrame()
    variaveis = data[variable].unique()
    for variavel in variaveis:
        dados = data[data[variable] == variavel]
        f_vendas, f_lucro, m_vendas, m_lucro, m_qtde, m_entrega, r_dias = rfm_variables(dados)
        new_row = {
            'referencia': variavel,
            'm_vendas': m_vendas,
            'm_lucro': m_lucro,
            'm_qtde': m_qtde,
            'm_entrega': m_entrega,
            'r_dias': r_dias,
            'f_vendas': f_vendas,
            'f_lucro': f_lucro
        }
        rfm = rfm.append(
            new_row,
            ignore_index = True
        )
    return rfm

In [6]:
def outliers_detection(orig, vars):
    sc_x = StandardScaler()
    datax = sc_x.fit_transform(vars)
    clf = KNN().fit(vars)
    outliers = clf.predict(vars)
    dataod = orig.copy()
    dataod['outlier'] = outliers
    return dataod

In [7]:
variaveis = ['m_vendas', 'm_lucro', 'm_qtde', 'm_entrega', 'r_dias', 'f_vendas', 'f_lucro']

In [8]:
## Aula 16 - Outliers
outliers_pais = fit_data(data, 'Country')
outliers_pais = outliers_pais.fillna(0)
outlier = outliers_pais[variaveis].fillna(0).copy()
outliers = outliers_detection(outliers_pais, outlier)
outliers

/tmp/ipykernel_3530/952877166.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
/tmp/ipykernel_3530/952877166.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
/tmp/ipykernel_3530/952877166.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
/tmp/ipykernel_3530/952877166.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
/tmp/ipykernel_3530/952877166.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
/tmp/ipykernel_3530/952877166.py:17: FutureWarning

,referencia,m_vendas,m_lucro,m_qtde,m_entrega,r_dias,f_vendas,f_lucro,outlier
0,Mexico,622590.62,102818.10,10011,67658.40,1.89,2644,2272,1
1,Colombia,81502.53,18798.05,1277,9411.77,9.32,360,350,0
2,Brazil,361106.42,30090.50,6148,38170.04,2.66,1599,1185,1
3,Nicaragua,149687.06,33401.44,2235,17155.17,5.30,614,600,0
4,Panama,51539.93,-17723.45,1426,5416.04,7.78,388,75,0
...,...,...,...,...,...,...,...,...,...
142,Ethiopia,850.62,290.16,18,81.54,358.67,7,7,0
143,Equatorial Guinea,150.51,44.46,3,9.38,0.00,3,3,0
144,Bahrain,669.18,12.84,6,72.76,0.00,2,2,0
145,Chad,1317.03,90.00,11,297.94,0.00,2,1,0


In [ ]:
outliers.to_feather('base/outliers_pais.feather')